In [1]:
import numpy as np
import os
import pandas as pd
import mne

from mne.preprocessing import ICA

In [7]:
#definisco il percorso iniziale comune 
home_path = os.path.abspath(os.getcwd())

#definisco una lista di nomi per salvare i file
savename=[]
for j in range(50):
    savename.append('paziente'+str(j+1)+'.fif')


for i in range(50):
    #scelgo indice del paziente che voglio studiare
    paziente= i+1
    #scrivo il percorso
    if paziente<10:
        filename = home_path+"\sub-0"+str(paziente)+"\eeg\sub-0"+str(paziente)+"_task-rsvp_eeg.vhdr"
    if paziente >= 10:
        filename = home_path+"\sub-"+str(paziente)+"\eeg\sub-"+str(paziente)+"_task-rsvp_eeg.vhdr"
    #importo eeg
    raw=mne.io.read_raw_brainvision(filename, preload= True)
    #Applico un filtro FIR tra 0.1 e 12 Hz per isolare le frequenze rilevanti
    raw_filtred=raw.filter(0.1, 12, method='fir')
    #nuovo riferimento dei segnali differenzial degli elettrodi
    raw_new_ref=raw_filtred.set_eeg_reference(ref_channels='average')
    #Il dataset ha un sample ratio di 1000 Hz, che è troppo alto, quindi faccio un resampling e lo porto a 250 Hz
    raw_resampled=raw_new_ref.copy().resample(sfreq=250)
    #taglio la frequenza 10Hz 
    raw_final_notch=raw_resampled.copy().notch_filter(freqs=10)
    #salvo il segnale prodotto
    raw_final_notch.save(savename[i], overwrite=True)
    

Extracting parameters from C:\Users\Maria Sole\Documents\TESI\sub-01\eeg\sub-01_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3035739  =      0.000 ...  3035.739 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 33001 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:   12.7s finished


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Setting up band-stop filter from 9.5 - 11 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 9.47
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 9.22 Hz)
- Upper passband edge: 10.53 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 10.78 Hz)
- Filter length: 1651 samples (6.604 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Writing C:\Users\Maria Sole\Documents\TESI\paziente1.fif


...

Closing C:\Users\Maria Sole\Documents\TESI\paziente49.fif
[done]
Extracting parameters from C:\Users\Maria Sole\Documents\TESI\sub-50\eeg\sub-50_task-rsvp_eeg.vhdr...
Setting channel info structure...
Reading 0 ... 3270959  =      0.000 ...  3270.959 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 12.00 Hz
- Upper transition bandwidth: 3.00 Hz (-6 dB cutoff frequency: 13.50 Hz)
- Filter length: 33001 samples (33.001 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 127 out of 127 | elapsed:   39.4s finished


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Setting up band-stop filter from 9.5 - 11 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 9.47
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 9.22 Hz)
- Upper passband edge: 10.53 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 10.78 Hz)
- Filter length: 1651 samples (6.604 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 127 out of 127 | elapsed:    4.3s finished
C:\Users\Maria Sole\AppData\Local\Temp\ipykernel_10440\3797255915.py:29: RuntimeWarning: This filename (C:\Users\Maria Sole\Documents\TESI\paziente50.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_final_notch.save(savename[i], overwrite=True)


Writing C:\Users\Maria Sole\Documents\TESI\paziente50.fif
Closing C:\Users\Maria Sole\Documents\TESI\paziente50.fif
[done]
